## Imports

In [ ]:
import asyncio

import pandas as pd
import geopandas as gpd
import aiohttp
from tqdm.asyncio import tqdm

from potentiel_solaire.constants import RESULTS_FOLDER
from potentiel_solaire.features.pvgis_api import async_get_potentiel_solaire_from_pvgis_api
from potentiel_solaire.features.solar_potential import calculate_installed_capacity

## Constants

In [ ]:
SEGMENTS_CHUNK_SIZE = 1000  # Nb de segments à traiter en simultané (asynchrone)
OUTPUT_CSV_PATH = RESULTS_FOLDER / "roof_segments_potential.csv"

## Récupération des résultats de la segmentation des toits

In [ ]:
# Lecture des segments de toit calculés avec le MNS
roof_segments_with_mns = pd.read_csv(RESULTS_FOLDER / "roof_segments_with_mns.csv")

## Préparation des params pour l'API pvgis
- Azimuth de l'algo de segmentation : 0 -> north / 90 -> east / 180 -> south / 270 -> west
- Azimuth attendue par l'api : -180 -> north / -90 -> east / 0 -> south / 90 -> west

In [ ]:
roof_segments_with_mns["peakpower"] = roof_segments_with_mns.apply(
    lambda segment: calculate_installed_capacity(segment.surface), axis=1
)
roof_segments_with_mns["optimalangles"] = roof_segments_with_mns.apply(
    lambda segment: 1 if segment.slope_bin_min == 0 else 0, axis=1
)
roof_segments_with_mns["angle"] = roof_segments_with_mns.apply(
    lambda segment: segment.slope if segment.slope_bin_min != 0 else 0, axis=1
)
roof_segments_with_mns["aspect"] = roof_segments_with_mns.apply(
    lambda segment: segment.azimut - 180 if segment.slope_bin_min != 0 else 0, axis=1
)

## Calcul du potentiel solaire pour chaque segment

In [ ]:
async def get_potentiel_solaire_segment(
    segment, 
    session: aiohttp.ClientSession,
    pbar: tqdm
):
    """Récupère le potentiel solaire pour un segment de toit de façon asynchrone.
    On souhaite retourner tous les attributs du segment, en plus du potentiel solaire calculé.
    """
    result = {
        **segment._asdict(),
        "solar_potential": await async_get_potentiel_solaire_from_pvgis_api(
            lat=segment.latitude,
            lon=segment.longitude,
            peakpower=segment.peakpower,
            optimalangles=segment.optimalangles,
            angle=segment.angle,
            aspect=segment.aspect,
            session=session,
            pbar=pbar
        )
    }

    del result["Index"]  # Supprimer l'index du résultat

    return result

In [ ]:
async def compute_solar_potential(
    segments: gpd.GeoDataFrame, 
    session: aiohttp.ClientSession,
    pbar: tqdm
) -> gpd.GeoDataFrame:
    """Calcule le potentiel solaire pour un chunk de segments de toit.
    Les résultats sont écrits dans un fichier CSV."""
    # Les appels à l'API pour calculer le potentiel solaire sont effectués de façon asynchrone
    tasks = [
        get_potentiel_solaire_segment(segment, session, pbar)
        for segment in segments.itertuples()
    ]

    # Récupérer les résultats de toutes les tâches et les transformer en DataFrame
    results = await asyncio.gather(*tasks, return_exceptions=True)
    df = pd.DataFrame(results)

    # Vérifier que tous les valeurs de potentiel solaire ont été calculées avec succès
    assert df["solar_potential"].notnull().all(), "Certaines valeurs de potentiel solaire sont manquantes."

    # Écrire le DataFrame dans un fichier CSV
    df.to_csv(OUTPUT_CSV_PATH, mode='a', header=not OUTPUT_CSV_PATH.exists(), index=False)

In [ ]:
# Calcul du potentiel solaire par chunk de segments de toits
with tqdm(total=len(roof_segments_with_mns), desc="Processing segments") as pbar:
    async with aiohttp.ClientSession() as session:
        segments_chunks = [
            roof_segments_with_mns[i : i + SEGMENTS_CHUNK_SIZE]
            for i in range(0, len(roof_segments_with_mns), SEGMENTS_CHUNK_SIZE)
        ]

        for chunk in segments_chunks:
            await compute_solar_potential(
                chunk,
                session=session,
                pbar=pbar
            )
